In [52]:
import csv
import numpy as np
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

#random seed 
np.random.seed(42)

dataset = 'data/landmarks.csv'
model_save_path = 'model/model.keras'


## Get the Data

In [53]:
xdata = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=range(1, 43)) # 21 landmarks, 2 coordinates each skip first col, gesture label
ydata = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=0)  # first column is the label

size = np.shape(xdata)
print(size)

(1372, 42)


Using Sequential Keras Model, high-level neural network API, built on top of Theano and Tensorflow. Sequential API mode comprises of linear pile of layers, allowing layer by layer configuration where each layer has exactly one input tensor and one ouptput tensor. Limited as unable to configure models with shared layers or have multiple inputs or outputs. sutible for this application as only input is relative landmark location, attained from Google's Mediapipe.

## Define the Model

In [54]:
model = keras.Sequential([
    keras.layers.Input(shape=(21*2, )), # 21 landmarks, each with x and y coordinates
    keras.layers.Dense(21, activation='relu'), # hidden layer, 21 neurons
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='relu'), # another hidden layer, 21 neurons
    keras.layers.Dense(6, activation='softmax'), # output layer, 2 gestures, softmax 
])

In [55]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 21)             │           903 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 21)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 10)             │           220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 6)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,189 (4.64 KB)

 Trainable params: 1,189 (4.64 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
checkpoint_callback = keras.callbacks.ModelCheckpoint(model_save_path, verbose=1, save_weights_only=False) # save best model during training
early_stopping_callback = keras.callbacks.EarlyStopping( patience=20, verbose=1) # stop training if no improvement for 20 epochs

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


## Train Model

In [61]:
# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.75, random_state=42)

# Train the model
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=250,
    batch_size=32,
    callbacks=[checkpoint_callback, early_stopping_callback]
)


Epoch 1/250
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9688 - loss: 0.1115
Epoch 1: saving model to model/model.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9661 - loss: 0.1241 - val_accuracy: 0.9728 - val_loss: 0.1406
Epoch 2/250
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9375 - loss: 0.1232
Epoch 2: saving model to model/model.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9693 - loss: 0.0926 - val_accuracy: 0.9718 - val_loss: 0.1398
Epoch 3/250
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.9375 - loss: 0.1731
Epoch 3: saving model to model/model.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9606 - loss: 0.1147 - val_accuracy: 0.9728 - val_loss: 0.1392
Epoch 4/250
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.9688 - loss: 0.1478
Epoch 4: saving model to model/model.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9830 - loss: 0.0929 - val_accuracy: 0.9718 - val_loss: 0.1404
Epoch 5/250
 1/11 ━━

## Evaluate

In [60]:
val_loss, val_accuracy = model.evaluate(x_val, y_val, batch_size=32)

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9750 - loss: 0.1034


In [59]:
predict_result = model.predict(np.array([x_val[0]]))
print(np.argmax(np.squeeze(predict_result))) 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
3
